Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [21]:
batch_size = 128
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
  
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    regularization = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    
    loss = loss + regularization_beta * regularization

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Prediction

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 21.230721
Minibatch accuracy: 8.6%
Validation accuracy: 7.9%
Minibatch loss at step 500: 1.936746
Minibatch accuracy: 84.4%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 2.032461
Minibatch accuracy: 75.8%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 1.078295
Minibatch accuracy: 82.8%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 1.026174
Minibatch accuracy: 85.2%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.845209
Minibatch accuracy: 83.6%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 0.883828
Minibatch accuracy: 78.9%
Validation accuracy: 80.6%
Test accuracy: 88.4%


In [99]:
batch_size = 128
hidden_layer_size = 1024
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():  
    tf_train_dataset = tf.placeholder(dtype = tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(dtype=tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # hidden layer
    
    weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset , weights_hidden) + biases_hidden)
    
    # output layer
    
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # L2 regularization
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) +
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))
    
    loss = loss + regularization_beta * regularizers
    
    # Optimizer
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)


In [100]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 440.270569
Minibatch accuracy: 10.9%
Validation accuracy: 17.3%
Minibatch loss at step 100: 192.295197
Minibatch accuracy: 75.0%
Validation accuracy: 74.9%
Minibatch loss at step 200: 185.565323
Minibatch accuracy: 77.3%
Validation accuracy: 76.4%
Minibatch loss at step 300: 178.398987
Minibatch accuracy: 75.8%
Validation accuracy: 77.9%
Minibatch loss at step 400: 173.255356
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Minibatch loss at step 500: 170.752136
Minibatch accuracy: 82.8%
Validation accuracy: 79.1%
Minibatch loss at step 600: 174.436508
Minibatch accuracy: 78.1%
Validation accuracy: 78.7%
Minibatch loss at step 700: 175.248047
Minibatch accuracy: 72.7%
Validation accuracy: 79.5%
Minibatch loss at step 800: 163.817749
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Minibatch loss at step 900: 163.836624
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 168.812790
Minibatch accuracy: 81.2%
Vali

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [97]:
batch_size = 10
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
  
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    regularization = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    
    loss = loss + regularization_beta * regularization

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Prediction

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [98]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.622482
Minibatch accuracy: 10.0%
Validation accuracy: 13.4%
Minibatch loss at step 500: 2.709300
Minibatch accuracy: 60.0%
Validation accuracy: 66.2%
Minibatch loss at step 1000: 3.998290
Minibatch accuracy: 70.0%
Validation accuracy: 70.1%
Minibatch loss at step 1500: 2.013970
Minibatch accuracy: 80.0%
Validation accuracy: 71.7%
Minibatch loss at step 2000: 3.965593
Minibatch accuracy: 60.0%
Validation accuracy: 73.1%
Minibatch loss at step 2500: 4.257615
Minibatch accuracy: 70.0%
Validation accuracy: 73.9%
Minibatch loss at step 3000: 2.837358
Minibatch accuracy: 70.0%
Validation accuracy: 74.7%
Test accuracy: 82.5%


In [95]:
batch_size = 20
hidden_layer_size = 1024
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():  
    tf_train_dataset = tf.placeholder(dtype = tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(dtype=tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # hidden layer
    
    weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset , weights_hidden) + biases_hidden)
    
    # output layer
    
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # L2 regularization
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) +
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))
    
    loss = loss + regularization_beta * regularizers
    
    # Optimizer
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)


In [96]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 407.031616
Minibatch accuracy: 15.0%
Validation accuracy: 20.2%
Minibatch loss at step 100: 238.735565
Minibatch accuracy: 60.0%
Validation accuracy: 73.0%
Minibatch loss at step 200: 233.739426
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Minibatch loss at step 300: 178.744400
Minibatch accuracy: 85.0%
Validation accuracy: 76.0%
Minibatch loss at step 400: 189.455109
Minibatch accuracy: 85.0%
Validation accuracy: 78.1%
Minibatch loss at step 500: 202.182892
Minibatch accuracy: 80.0%
Validation accuracy: 75.5%
Minibatch loss at step 600: 196.436584
Minibatch accuracy: 85.0%
Validation accuracy: 78.4%
Minibatch loss at step 700: 167.498657
Minibatch accuracy: 85.0%
Validation accuracy: 77.3%
Minibatch loss at step 800: 193.763031
Minibatch accuracy: 80.0%
Validation accuracy: 77.2%
Minibatch loss at step 900: 149.323929
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 162.744598
Minibatch accuracy: 85.0%
Val

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [93]:
import math

batch_size = 20
hidden_layer_size = 2048 # Doubled because half of the results are discarded
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size], 
                           stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(tf.nn.dropout(hidden, 0.5), weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [94]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31.172670
Minibatch accuracy: 5.0%
Validation accuracy: 24.8%
Minibatch loss at step 100: 27.626163
Minibatch accuracy: 80.0%
Validation accuracy: 73.1%
Minibatch loss at step 200: 46.231911
Minibatch accuracy: 60.0%
Validation accuracy: 74.8%
Minibatch loss at step 300: 20.550724
Minibatch accuracy: 80.0%
Validation accuracy: 76.9%
Minibatch loss at step 400: 23.257557
Minibatch accuracy: 85.0%
Validation accuracy: 78.6%
Minibatch loss at step 500: 43.699242
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 600: 25.478069
Minibatch accuracy: 80.0%
Validation accuracy: 78.0%
Minibatch loss at step 700: 20.097382
Minibatch accuracy: 80.0%
Validation accuracy: 78.8%
Minibatch loss at step 800: 18.852102
Minibatch accuracy: 85.0%
Validation accuracy: 78.8%
Minibatch loss at step 900: 6.166813
Minibatch accuracy: 95.0%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 37.243134
Minibatch accuracy: 90.0%
Validation accura

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [84]:
batch_size = 128
hidden_layer_size = 1024 * 2
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():
    # data
    
    tf_train_dataset = tf.placeholder(dtype=tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(dtype=tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # Hidden layer
    
    weights_hidden_1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer_size]
        ,stddev=1 / math.sqrt(float(image_size*image_size))))
    biases_hidden_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden_1) + biases_hidden_1)
                                      
    weights_hidden_2 = tf.Variable(tf.truncated_normal([hidden_layer_size,hidden_layer_size]
        ,stddev=1 / math.sqrt(float(image_size*image_size))))
    biases_hidden_2 = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_2 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_1, 0.5), weights_hidden_2) + biases_hidden_2)
    
                                   
    # Output layer
    
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]
                                 ,stddev=1/ math.sqrt(float(image_size*image_size))))
    biases_output = tf.Variable(tf.zeros([num_labels]))
                                   
                                   
    # Training model

    logits = tf.matmul(tf.nn.dropout(hidden_2,0.5), weights_output) + biases_output
                                   
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # regularization
                                
    regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) + 
                    tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) +
                    tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
                                   
    # Optimizer
                                   
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
                                   
    # Predictions
                                   
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
    hidden_validation_2 = tf.nn.relu(tf.matmul(hidden_validation_1, weights_hidden_2) + biases_hidden_2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation_2, weights_output) + biases_output)
    
    hidden_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
    hidden_prediction_2 = tf.nn.relu(tf.matmul(hidden_prediction_1, weights_hidden_2) + biases_hidden_2)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction_2, weights_output) + biases_output)                            
    
    
                            
    

In [92]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.101059
Minibatch accuracy: 8.6%
Validation accuracy: 22.9%
Minibatch loss at step 100: 2.022937
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 200: 2.000785
Minibatch accuracy: 79.7%
Validation accuracy: 82.5%
Minibatch loss at step 300: 1.982693
Minibatch accuracy: 78.1%
Validation accuracy: 83.2%
Minibatch loss at step 400: 1.675238
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 500: 1.604136
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 600: 1.725059
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 700: 1.685758
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 800: 1.413249
Minibatch accuracy: 86.7%
Validation accuracy: 85.5%
Minibatch loss at step 900: 1.631237
Minibatch accuracy: 80.5%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 1.463517
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
